## Import the Libraries and Modules

In [74]:
import os
import sys

In [75]:
module_path = os.path.abspath(os.path.join("../utils"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/cram/dev/using-cognite-python-sdk/utils


In [13]:
import auth as cauth

## Create the Cognite Client

In [14]:
c = cauth.create_cognite_client(method="client-secret")

/home/cram/dev/using-cognite-python-sdk/.venv/lib/python3.8/site-packages/cognite/client/_cognite_client.py:85: UserWarning: You are using version 2.56.0 of the SDK, however version 2.56.1 is available. Upgrade or set the environment variable 'COGNITE_DISABLE_PYPI_VERSION_CHECK' to suppress this warning.
  self._config = ClientConfig(


## Create various resource types

Create timeseries, datasets, labels, events etc.

**Code Pattern**

`client.<cdf_resource_type>.create()`

Dataset

In [15]:
from cognite.client.data_classes import DataSet
data_sets = [DataSet(name="1st level",external_id="first_level"), 
            DataSet(name="2nd level",external_id="second_level")]
res = c.data_sets.create(data_sets)

In [16]:
c.data_sets.list()

,external_id,name,metadata,write_protected,id,created_time,last_updated_time,description
0,test_dataset8,test_dataset8,{},False,404471063348808,1657143733168,1657143733168,NaN
1,first_level,1st level,{},False,1900770456801810,1657199464830,1657199464830,NaN
2,NaN,my_test_dataset,{},False,1929610961715132,1657190954116,1657190954116,NaN
3,test_dataset2,test_dataset2,{},False,2528428316884654,1657143732740,1657143732740,NaN
4,NaN,2nd level,{'archived': 'true'},False,2697380900418431,1657198128127,1657198654684,NaN
5,test_dataset1,test_dataset1,{},False,2748583307904895,1657143732665,1657143732665,NaN
6,test_dataset4,test_dataset4,{},False,4198465485109543,1657143732881,1657143732881,NaN
7,NaN,my_test_dataset,{},False,4397486057907956,1657198128258,1657198128258,NaN
8,test_dataset7,test_dataset7,{},False,4750475368646754,1657143733093,1657143733093,NaN
9,test_dataset3,test_dataset3,{},False,5087716596235727,1657143732811,1657143732811,NaN


In [17]:
res = c.data_sets.create([DataSet(name="my_example_dataset",external_id="my_example_dataset")])
res

,external_id,name,write_protected,id,created_time,last_updated_time
0,my_example_dataset,my_example_dataset,False,1399248851345249,1657199513845,1657199513845


In [18]:
test_ds_id = 1399248851345249

Labels

In [19]:
from cognite.client.data_classes import LabelDefinition
labels = [LabelDefinition(external_id="ROTATING_EQUIPMENT", name="Rotating equipment",data_set_id=test_ds_id),
LabelDefinition(external_id="PUMP", name="pump",data_set_id=test_ds_id)]
res = c.labels.create(labels)

In [20]:
c.labels.list()

,external_id,name,created_time,data_set_id
0,ROTATING_EQUIPMENT,Rotating equipment,1657199620472,1399248851345249
1,REGION,REGION,1657141671698,<NA>
2,COUNTRY,COUNTRY,1657141671698,<NA>
3,PUMP,pump,1657199620472,1399248851345249
4,BALTIC,BALTIC,1657143421635,<NA>
5,NORDIC,NORDIC,1657142354433,<NA>


Assets

In [21]:
from cognite.client.data_classes import Asset
assets = [Asset(name="asset1",external_id="asset1",data_set_id=test_ds_id), 
            Asset(name="asset2",external_id="asset2",data_set_id=test_ds_id)]
res = c.assets.create(assets)

In [22]:
c.assets.list(data_set_ids=[test_ds_id])

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id
0,asset1,asset1,1399248851345249,{},1129940512190402,1657199689894,1657199689894,1129940512190402
1,asset2,asset2,1399248851345249,{},2856135849713652,1657199689894,1657199689894,2856135849713652


Timeseries

In [23]:
from cognite.client.data_classes import TimeSeries
ts = c.time_series.create(TimeSeries(name="my_ts",external_id = "my_ts",data_set_id=test_ds_id ))

In [24]:
ts

,value
id,5258405260776053
external_id,my_ts
name,my_ts
is_string,False
is_step,False
security_categories,[]
data_set_id,1399248851345249
created_time,1657199813970
last_updated_time,1657199813970


Sequences

In [26]:
from cognite.client.data_classes import Sequence
column_def = [{"valueType":"STRING","externalId":"user","description":"some description"},
             {"valueType":"DOUBLE","externalId":"amount"}]
seq = c.sequences.create(Sequence(external_id="my_test_sequence",name="my test sequence", columns=column_def,data_set_id=test_ds_id ))

Files metadata

In [27]:
from cognite.client.data_classes import FileMetadata
file_metadata = FileMetadata(name="MyFile",external_id="my_file",data_set_id=test_ds_id)
res = c.files.create(file_metadata)

In [28]:
res

(<cognite.client.data_classes.files.FileMetadata at 0x7f1d0c5ee7f0>,
 'https://westeurope-1.cognitedata.com/api/v1/files/storage/cognite/6559848865888927/1500530369065916/MyFile?sig=tXSvJUSmosmV0WHQ1el4tHJH0ieabRwNHLbWSlwhNj4%3D&se=2022-07-14T13%3A21%3A07Z&sv=2019-02-02&sp=cw&sr=b')

Relationships

In [30]:
from cognite.client.data_classes import Relationship
flowrel1 = Relationship(external_id="flow_12_test", source_external_id="asset1", source_type="asset", target_external_id="asset2", target_type="asset", confidence=0.1, data_set_id=test_ds_id)
flowrel2 = Relationship(external_id="flow_21_test", source_external_id="asset2", source_type="asset", target_external_id="asset1", target_type="asset", confidence=0.1, data_set_id=test_ds_id)
res = c.relationships.create([flowrel1,flowrel2])

In [34]:
c.relationships.list()

,external_id,source_external_id,source_type,target_external_id,target_type,confidence,data_set_id,created_time,last_updated_time,labels
0,norway_neighbours_1,Norway,asset,Sweden,asset,1.0,5548572851312583,1657144744739,1657144744739,[]
1,norway_neighbours_2,Norway,asset,Finland,asset,1.0,5548572851312583,1657144744739,1657144744739,[]
2,flow_12,asset1,asset,asset2,asset,0.1,1929610961715132,1657191250639,1657191250639,[]
3,flow_21,asset2,asset,asset1,asset,0.1,1929610961715132,1657191250639,1657191250639,[]
4,flow_12_test,asset1,asset,asset2,asset,0.1,1399248851345249,1657200238497,1657200238497,[]
5,flow_21_test,asset2,asset,asset1,asset,0.1,1399248851345249,1657200238497,1657200238497,[]


Create Asset with Labels

In [35]:
from cognite.client.data_classes import Label
asset = Asset(name="my_test_pump", labels=[Label(external_id="PUMP")],data_set_id=test_ds_id)
res = c.assets.create(asset)

In [36]:
res

,value
name,my_test_pump
data_set_id,1399248851345249
labels,[{'externalId': 'PUMP'}]
id,7363043841868644
created_time,1657200440035
last_updated_time,1657200440035
root_id,7363043841868644


Create Asset Hierarchy

In [38]:
assets_list = [Asset(external_id="root", name="Root",data_set_id=test_ds_id),
          Asset(external_id="child1", parent_external_id="root", name="child1",data_set_id=test_ds_id), 
          Asset(external_id="child2", parent_external_id="root", name="child2",data_set_id=test_ds_id),
          Asset(external_id="child_of_child1", parent_external_id="child1", name="child_of_child1",data_set_id=test_ds_id)]
res = c.assets.create_hierarchy(assets_list)

In [39]:
c.assets.retrieve_subtree(external_id="root")

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id,parent_id,parent_external_id
0,root,Root,1399248851345249,{},4299328479638485,1657200692566,1657200692566,4299328479638485,<NA>,NaN
1,child2,child2,1399248851345249,{},2994057977739744,1657200692566,1657200692566,4299328479638485,4299328479638485,root
2,child1,child1,1399248851345249,{},6709553116468444,1657200692566,1657200692566,4299328479638485,4299328479638485,root
3,child_of_child1,child_of_child1,1399248851345249,{},6986429626685430,1657200692566,1657200692566,4299328479638485,6709553116468444,child1


Upload files and folders

Single file

In [40]:
res = c.files.upload("myfiles", name="my_file_test1.txt")

In [42]:
c.files.list(limit=-1)

,name,asset_ids,id,uploaded,uploaded_time,created_time,last_updated_time,source,mime_type,data_set_id,external_id
0,PH-ME-P-0004-001.pdf,[8628084799352049],751071820891431,True,1.657139e+12,1657139202305,1657186316980,NaN,NaN,<NA>,NaN
1,my_file_test1.txt,NaN,979093307589937,False,NaN,1657200941592,1657200941592,NaN,NaN,<NA>,NaN
2,PH-ME-P-0156-002.pdf,[8628084799352049],1079820226552851,False,NaN,1657138728081,1657186317119,Discovery,application/pdf,7129880323294884,NaN
3,PH-ME-P-0156-001.pdf,[8628084799352049],1183954407347440,False,NaN,1657138727341,1657186317296,Discovery,application/pdf,7129880323294884,NaN
4,PH-ME-P-0153-001.pdf,[8628084799352049],1191573487368640,True,1.657139e+12,1657139202316,1657186317414,NaN,NaN,<NA>,NaN
5,MyFile,NaN,1500530369065916,False,NaN,1657200067669,1657200067669,NaN,NaN,1399248851345249,my_file
6,PH-ME-P-0003-001.pdf,[8628084799352049],1518761967810752,False,NaN,1657138728156,1657186317521,Discovery,application/pdf,7129880323294884,NaN
7,PH-25578-P-4110006-001.pdf,[8628084799352049],1747542106535044,True,1.657139e+12,1657139204394,1657186317619,NaN,NaN,<NA>,NaN
8,test1.txt,NaN,1893188072779304,False,NaN,1657191529154,1657191529154,NaN,NaN,<NA>,NaN
9,PH-ME-P-0151-001.pdf,[8628084799352049],2461583640351869,False,NaN,1657138728710,1657186317755,Discovery,application/pdf,7129880323294884,NaN


All files in a directory

In [43]:
res = c.files.upload("files_to_upload")

## Update various resource types
**Code Pattern**

`client.<cdf_resource_type>.update()`

Full Update

In [45]:
c.data_sets.list()

,external_id,name,metadata,write_protected,id,created_time,last_updated_time,description
0,test_dataset8,test_dataset8,{},False,404471063348808,1657143733168,1657143733168,NaN
1,my_example_dataset,my_example_dataset,{},False,1399248851345249,1657199513845,1657199513845,NaN
2,first_level,1st level,{},False,1900770456801810,1657199464830,1657199464830,NaN
3,NaN,my_test_dataset,{},False,1929610961715132,1657190954116,1657190954116,NaN
4,test_dataset2,test_dataset2,{},False,2528428316884654,1657143732740,1657143732740,NaN
5,NaN,2nd level,{'archived': 'true'},False,2697380900418431,1657198128127,1657198654684,NaN
6,test_dataset1,test_dataset1,{},False,2748583307904895,1657143732665,1657143732665,NaN
7,test_dataset4,test_dataset4,{},False,4198465485109543,1657143732881,1657143732881,NaN
8,NaN,my_test_dataset,{},False,4397486057907956,1657198128258,1657198128258,NaN
9,test_dataset7,test_dataset7,{},False,4750475368646754,1657143733093,1657143733093,NaN


In [46]:
data_set = c.data_sets.retrieve(id=1399248851345249)
data_set

,value
external_id,my_example_dataset
name,my_example_dataset
write_protected,False
id,1399248851345249
created_time,1657199513845
last_updated_time,1657199513845


In [47]:
data_set.description = "This is a Example Dataset"
res = c.data_sets.update(data_set)

In [48]:
res

,value
external_id,my_example_dataset
name,my_example_dataset
description,This is a Example Dataset
write_protected,False
id,1399248851345249
created_time,1657199513845
last_updated_time,1657201469805


Partial Update

In [49]:
from cognite.client.data_classes import DataSetUpdate
my_update = DataSetUpdate(id=1399248851345249).description.set("New description with Partial Update")#.metadata.remove(["key"])
res = c.data_sets.update(my_update)

In [50]:
res

,value
external_id,my_example_dataset
name,my_example_dataset
description,New description with Partial Update
write_protected,False
id,1399248851345249
created_time,1657199513845
last_updated_time,1657201639823


In [51]:
c.assets.list(root=True)

,external_id,name,data_set_id,metadata,id,created_time,last_updated_time,root_id,description,source,labels
0,asset1,asset1,1399248851345249,{},1129940512190402,1657199689894,1657199689894,1129940512190402,NaN,NaN,NaN
1,6687602007296940,Aker BP,<NA>,{},2075231145010555,1652690345473,1652690345473,2075231145010555,Aker BP,petro-tech-staging,NaN
2,houston.00. Support systems.Reverse osmosis,Reverse osmosis,<NA>,"{'_replicatedInternalId': '1536954437306151', ...",2436442502710362,1652690345473,1652690345473,2436442502710362,NaN,petro-tech-staging,NaN
3,world,Global,5548572851312583,{},2588735622961188,1657141408621,1657141436065,2588735622961188,World asset used as root for all countries,NaN,NaN
4,asset2,asset2,1399248851345249,{},2856135849713652,1657199689894,1657199689894,2856135849713652,NaN,NaN,NaN
5,4093404255107247,Vulkan Control Room,<NA>,{'source': 'Point cloud model of control room ...,3326391988716762,1652690345473,1652690345473,3326391988716762,NaN,petro-tech-staging,NaN
6,root,Root,1399248851345249,{},4299328479638485,1657200692566,1657200692566,4299328479638485,NaN,NaN,NaN
7,NaN,my_pump,1929610961715132,{},7097748601985097,1657191300482,1657191300482,7097748601985097,NaN,NaN,[{'externalId': 'PUMP'}]
8,NaN,my_test_pump,1399248851345249,{},7363043841868644,1657200440035,1657200440035,7363043841868644,NaN,NaN,[{'externalId': 'PUMP'}]


In [52]:
from cognite.client.data_classes import AssetUpdate
my_update = AssetUpdate(external_id="asset1").description.set("New description using AssetUpdate")#.metadata.add({"key": "value"})
res1 = c.assets.update(my_update)

In [53]:
res1

,value
external_id,asset1
name,asset1
description,New description using AssetUpdate
data_set_id,1399248851345249
id,1129940512190402
created_time,1657199689894
last_updated_time,1657201843630
root_id,1129940512190402


Partial Update

In [ ]:
another_update = AssetUpdate(id=1).description.set(None)
res2 = c.assets.update(another_update)



---



In [ ]:
res = c.time_series.retrieve(id=1)
res.description = "New description"
res = c.time_series.update(res)

In [ ]:
res = c.sequences.retrieve(id=1)
res.description = "New description"
res = c.sequences.update(res)

In [ ]:
event = c.events.retrieve(id=1)
event.description = "New description"
res = c.events.update(event)

In [ ]:
file_metadata = c.files.retrieve(id=1)
file_metadata.description = "New description"
res = c.files.update(file_metadata)

In [ ]:
rel = c.relationships.retrieve(external_id="flow1")
rel.confidence = 0.75
res = c.relationships.update(rel)

## Insert the data in CDF

**Code Pattern**
`client.<cdf_resource_type>.insert()`

### Insert the Datapoints/Rows

In [56]:
c.time_series.list(external_id_prefix="my_")

,id,external_id,name,is_string,metadata,is_step,security_categories,data_set_id,created_time,last_updated_time
0,5258405260776053,my_ts,my_ts,False,{},False,[],1399248851345249,1657199813970,1657199813970


In [57]:
from datetime import datetime
# with datetime objects
datapoints = [(datetime(2018,1,1), 1000), (datetime(2018,1,2), 2000)]
c.datapoints.insert(datapoints, id=5258405260776053)

In [58]:
# with ms since epoch
datapoints = [(150000000000, 1000), (160000000000, 2000)]
c.datapoints.insert(datapoints, id=5258405260776053)

In [60]:
c.datapoints.retrieve(external_id="my_ts",start=datetime(2018,1,1),end=datetime(2019,1,2))

,my_ts
2018-01-01,1000.0
2018-01-02,2000.0


In [61]:
c.sequences.list()

,id,external_id,metadata,columns,created_time,last_updated_time,data_set_id,name,description
0,2357839693666680,my_sequence,{},"[{'externalId': 'user', 'description': 'some d...",1657191142856,1657191142856,1929610961715132,NaN,NaN
1,7408721814460392,world_table,{},"[{'externalId': 'name', 'valueType': 'STRING',...",1657147108981,1657147401929,5548572851312583,world_table,Table containing data about countries of world
2,8726735623634190,my_test_sequence,{},"[{'externalId': 'user', 'description': 'some d...",1657199968405,1657199968405,1399248851345249,my test sequence,NaN


In [63]:
c.sequences.retrieve(id=2357839693666680).columns

[{'externalId': 'user',
  'description': 'some description',
  'valueType': 'STRING',
  'metadata': {},
  'createdTime': 1657191142856,
  'lastUpdatedTime': 1657191142856},
 {'externalId': 'amount',
  'valueType': 'DOUBLE',
  'metadata': {},
  'createdTime': 1657191142856,
  'lastUpdatedTime': 1657191142856}]

In [76]:
data = [(1, ['user1',3.14]), (2, ['user2',2.72]) ]
c.sequences.data.insert(column_external_ids=["user","amount"], rows=data, id=2357839693666680)

In [68]:
c.sequences.data.retrieve_dataframe(id=2357839693666680,start=0,end=100)

,user,amount
1,user1,3.14
2,user2,2.72


### Insert the Dataframe

In [70]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [71]:
ts_id = 5258405260776053
x = pd.DatetimeIndex([datetime(2018, 1, 1) + timedelta(days=d) for d in range(100)])
y = np.random.normal(0, 1, 100)
df = pd.DataFrame({ts_id: y}, index=x)
df

,5258405260776053
2018-01-01,1.201658
2018-01-02,-1.440902
2018-01-03,0.073946
2018-01-04,1.349312
2018-01-05,1.099516
...,...
2018-04-06,0.980378
2018-04-07,2.486425
2018-04-08,1.100117
2018-04-09,-0.019817


In [72]:
c.datapoints.insert_dataframe(df)

In [ ]:
c.sequences.data.insert_dataframe(df*2, id=123)

### Insert the datapoints in multiple timeseries

In [73]:
datapoints = []
# with datetime objects and id
datapoints.append({"id": 1, "datapoints": [(datetime(2018,1,1), 1000), (datetime(2018,1,2), 2000)]})
# with ms since epoch and externalId
datapoints.append({"externalId": "my_ts", "datapoints": [(150000000000, 1000), (160000000000, 2000)]})
datapoints

[{'id': 1,
  'datapoints': [(datetime.datetime(2018, 1, 1, 0, 0), 1000),
   (datetime.datetime(2018, 1, 2, 0, 0), 2000)]},
 {'externalId': 'my_ts',
  'datapoints': [(150000000000, 1000), (160000000000, 2000)]}]

In [ ]:
c.datapoints.insert_multiple(datapoints)